# compute functional connectomes from timeseries and save as csv


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
from nilearn.connectome import ConnectivityMeasure
import glob
import os
import pickle


In [ ]:
wd = os.getcwd()
wd = (wd[:-10]+'/')
path = wd+'216/'
path

1) schaefer atals:

In [ ]:

# Read dictionary pkl file
with open(path+'/data/ukb_data_to_process/func', 'rb') as fp:
    data = pickle.load(fp)
for key, value in data.items():
    df = value
    df.to_csv(f'{path}/HC/time_series/{key}-sch.csv', index=True) 

2) tian atlas

In [ ]:
# Read dictionary pkl file
with open(path+'/ukb_data_to_process/fbc', 'rb') as fp:
    data = pickle.load(fp)
for key, value in data.items():
    df = value
    df.to_csv(f'{path}/HC/time_series/{key}-tian.csv', index=True)   

concat the time series from cortical and subcortical atlas

In [ ]:


# List of CSV files in each directory
list_sch = glob.glob(path+'/HC/time_series/*-sch.csv')
list_msaI = glob.glob(path+'/HC/time_series/*-tian.csv')

# Sort files to ensure proper ordering
list_sch.sort()
list_msaI.sort()

# Iterate through the files and concatenate them
for file1, file2 in zip(list_sch, list_msaI):
    
    # Read CSV files
    df1 = pd.read_csv(file1)
    df2 = pd.read_csv(file2)
    
    # Concatenate dataframes
    concatenated_df = pd.concat([df1, df2], axis=0)
    
    # Write concatenated dataframe to a new CSV file
    concatenated_df.to_csv(f'{path}/HC/time_series/{file1[-15:-8]}_whole_brain.csv', index=False) 
    
    print(f'Concatenated file created for {file1} and {file2}')

### check for missing valus
replacing their values with group avg

In [ ]:
list_file = glob.glob(path+'/HC/time_series/*_whole_brain.csv')
my_data = pd.DataFrame()
for i in list_file:
    df = pd.read_csv(i, index_col=0)
    missing_values = df.isna().sum().sum()
    if missing_values != 0:
        print(i[-23:-16])
        index_with_missing_values = df[df.isna().any(axis=1)].index
        print("Index labels with missing values:", index_with_missing_values)

### If missing:
edit to eid of subject with missing

In [ ]:
eid='1234567'  # Example eid with missing values
df = pd.read_csv(path+'/HC/time_series/'+eid+'_whole_brain.csv', index_col=0)
column_means = df.mean()
# Replace missing values in rows with the calculated averages
for col in df.columns:
    mask = df[col].isnull()  # Mask for missing values in the column
    df.loc[mask, col] = column_means[col]
df.to_csv(path+'/HC/time_series/'+eid+'_whole_brain.csv', index=True)
df = pd.read_csv(path+'/HC/time_series/'+eid+'_whole_brain.csv', index_col=0)
df.loc['7Networks_LH_Cont_Cing_2']

### Compute corr

Correlation

In [20]:
list_file = glob.glob(path+'/HC/time_series/*_whole_brain.csv')
my_data = pd.DataFrame()
for i in list_file:
    df = pd.read_csv(i, index_col=0)
    time_series=df.values
    time_series=np.transpose(time_series)
    labels = df.index.values.tolist()
    correlation_measure = ConnectivityMeasure(
        kind="correlation",
        standardize=True,
    )
    correlation_matrix = correlation_measure.fit_transform([time_series])[0]
    corre =  pd.DataFrame(correlation_matrix)
    leb =  pd.DataFrame(labels)
    # Save DataFrame to CSV
    corre.to_csv(f'{path}/HC/func/{i[-23:-16]}-cor.csv', index=False, header=False)
###    my_data = pd.concat([my_data, corre])

leb.to_csv(f'{path}/HC/leb/sch200-msai-leb.csv', index=False,header=False)

partial correlation

In [21]:
list_file = glob.glob(path+'/HC/time_series/*_whole_brain.csv')
my_data = pd.DataFrame()
for i in list_file:
    df = pd.read_csv(i, index_col=0)
    time_series=df.values
    time_series=np.transpose(time_series)
    labels = df.index.values.tolist()
    correlation_measure = ConnectivityMeasure(
        kind="partial correlation",
        standardize=True,
    )
    correlation_matrix = correlation_measure.fit_transform([time_series])[0]
    corre =  pd.DataFrame(correlation_matrix)
    leb =  pd.DataFrame(labels)
    # Save DataFrame to CSV
    corre.to_csv(f'{path}/HC/func/{i[-23:-16]}-pcor.csv', index=False, header=False)